In [1]:
import cv2
import datetime
import pandas as pd
import numpy as np
import dlib
import os
from matplotlib import pyplot as plt
import skvideo.io
import matplotlib as mpl   
import time

In [2]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [9]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
count = 0
df_x = pd.DataFrame()
df_y = pd.DataFrame()
num = int(input('운동 횟수를 입력해주세요'))
playtime = int(input('각 운동당 소요시간을 입력해주세요 (테스트용, 단위 sec)'))


webcam = cv2.VideoCapture(0)

if not webcam.isOpened():
    print("Could not open webcam")
    exit()

while webcam.isOpened():
    status, frame = webcam.read()

    if status:
        cv2.imshow("test", frame)

    key = cv2.waitKey(1)
    # if key & 0xFF == ord('q'):
    #     break

    if key & 0xFF == ord('s'):
        for i2 in range (num):
            playtime1 = int(str(round(time.time()))[-4:])
            playsum = playtime1 + playtime
            while True:
                status, frame = webcam.read()
                playtime2 = int(str(round(time.time()))[-4:])
                if playtime2 >= playsum:
                    break
                time.sleep(0.5)
                print('Capture Start')
                ## resize할 비율 구하기
                img = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                r = 300. / img.shape[1]
                dim = (300, int(img.shape[0] * r))

                resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
                
                rects = detector(resized, 1)
                for i, rect in enumerate(rects):
                    ## 찾은 얼굴의 박스좌표
                    l = rect.left()
                    t = rect.top()
                    b = rect.bottom()
                    r = rect.right()
                    ## facial landmark 찾기
                    shape = predictor(resized, rect)
                    ## facial landmark를 빨간색 점으로 찍어서 표현
                    x_list = []
                    y_list = []
                    for j in range(68):
                        x, y = shape.part(j).x, shape.part(j).y
                        cv2.circle(resized, (x, y), 1, (0, 0, 255), -1)
                        x_list.append(x)
                        y_list.append(y)
                        cv2.rectangle(resized, (l, t), (r, b), (0, 255, 0), 2)
                        result = cv2.putText(resized, str(j), (x, y), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 0.3, (0, 255, 0))
                        ## 얼굴이 있는 부분을 박스쳐주기
                    count += 1
                    df_x['face' + str(count)] = x_list
                    df_y['face' + str(count)] = y_list
                    createFolder('test_' + str(i2))
                    file = 'test_' + str(i2) + '\\' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S%f") + '.jpg'
                    cv2.imwrite(file, result)
                    print(file, ' saved')
        createFolder('df')
        df_x.to_csv('df\\test_dfx.csv')
        df_y.to_csv('df\\test_dfy.csv')
    if key & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows()

Capture Start
test_0\20221205_145100531342.jpg  saved
Capture Start
test_0\20221205_145101097084.jpg  saved
Capture Start
test_0\20221205_145101664565.jpg  saved
Capture Start
test_0\20221205_145102228058.jpg  saved
Capture Start
test_0\20221205_145102794543.jpg  saved
Capture Start
test_1\20221205_145103361542.jpg  saved
Capture Start
test_1\20221205_145103944440.jpg  saved
Capture Start
test_1\20221205_145104512920.jpg  saved
Capture Start
test_1\20221205_145105165178.jpg  saved
Capture Start
test_1\20221205_145105730106.jpg  saved
Capture Start
test_2\20221205_145106360438.jpg  saved
Capture Start
test_2\20221205_145106932573.jpg  saved
Capture Start
Capture Start
Capture Start
test_2\20221205_145108686232.jpg  saved


In [6]:
df_x

,face1,face2,face3,face4,face5,face6,face7,face8,face9,face10,face11,face12,face13,face14,face15,face16
0,101,100,100,105,103,104,108,109,107,109,104,102,105,105,108,110
1,102,102,101,105,104,105,108,110,109,110,104,103,106,107,108,110
2,103,103,103,107,106,106,109,111,110,111,106,105,108,108,109,111
3,106,105,105,109,109,109,111,113,113,114,109,108,110,111,112,113
4,110,109,110,113,113,112,114,118,118,118,113,111,114,116,116,117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,148,148,149,148,144,140,147,157,167,160,145,143,148,153,155,156
64,157,157,158,158,155,151,157,166,174,168,155,153,158,163,165,165
65,148,148,150,148,145,140,147,157,167,160,146,143,148,154,156,156
66,144,144,146,144,141,136,143,153,163,156,141,139,144,149,152,152
